In [ ]:
!pip install torch torchvision torchaudio transformers datasets accelerate -q
!pip install transformers==4.40.2 datasets accelerate

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
from torch.optim import AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [ ]:
dataset = load_dataset("glue", "sst2")

train_dataset = dataset["train"]
val_dataset = dataset["validation"]

teacher_model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(teacher_model_name)

def tokenize_fn(batch):
    return tokenizer(batch["sentence"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_fn, batched=True)
val_dataset = val_dataset.map(tokenize_fn, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

sst2/train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

sst2/validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

sst2/test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

In [ ]:

teacher = AutoModelForSequenceClassification.from_pretrained(teacher_model_name, num_labels=2).to(device)
teacher.eval()

student_model_name = "google/bert_uncased_L-2_H-128_A-2"
student = AutoModelForSequenceClassification.from_pretrained(student_model_name, num_labels=2).to(device)

optimizer = AdamW(student.parameters(), lr=5e-5)
num_epochs = 2
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler("linear", optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


config.json:   0%|          | 0.00/382 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def distillation_loss(student_logits, teacher_logits, labels, T=2.0, alpha=0.7):

    distill_loss = F.kl_div(
        F.log_softmax(student_logits / T, dim=-1),
        F.softmax(teacher_logits / T, dim=-1),
        reduction="batchmean"
    ) * (T * T)

    ce_loss = F.cross_entropy(student_logits, labels)

    return alpha * distill_loss + (1 - alpha) * ce_loss


In [ ]:
from tqdm import tqdm

for epoch in range(num_epochs):
    student.train()
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
    for batch in progress_bar:
        batch = {k: v.to(device) for k, v in batch.items()}

        with torch.no_grad():
            teacher_outputs = teacher(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])

        student_outputs = student(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])

        loss = distillation_loss(student_outputs.logits, teacher_outputs.logits, batch["label"])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        progress_bar.set_postfix({"loss": loss.item()})


Epoch 2: 100%|██████████| 2105/2105 [04:14<00:00,  8.26it/s, loss=0.168]


In [ ]:
from sklearn.metrics import accuracy_score

def evaluate(model, dataloader):
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])
            preds.extend(torch.argmax(outputs.logits, dim=-1).cpu().numpy())
            labels.extend(batch["label"].cpu().numpy())
    return accuracy_score(labels, preds)

teacher_acc = evaluate(teacher, val_loader)
student_acc = evaluate(student, val_loader)

print(f"Teacher Accuracy: {teacher_acc*100:.2f}%")
print(f"Student Accuracy: {student_acc*100:.2f}%")


Teacher Accuracy: 49.08%
Student Accuracy: 81.08%


In [ ]:
student.save_pretrained("distilled-student")
tokenizer.save_pretrained("distilled-student")
print("Student model saved")


Student model saved
